In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [2]:
#import db
db=pd.read_csv("student-por.csv", delimiter=";")
print(db.shape)
db.head()

(649, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [3]:
#G3 stünununu 10 üzeri puanları başarılı(1), altında kalan puanları başarısız(0) olacak şekilde düzenliyoruz
df=db.copy()
df['PASS']=df['G3'].map(lambda x:0 if x<10 else 1)
df.drop(['G3'],axis=1,inplace=True)
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,PASS
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,1
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,1
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,0
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,1


In [4]:
#veri sayısallaştırma (https://medium.com/datarunner/veri-%C3%B6n-i%CC%87%C5%9Fleme-kategorik-verilerin-say%C4%B1salla%C5%9Ft%C4%B1r%C4%B1lmas%C4%B1-6ba7e78c1be1)
from sklearn.preprocessing import LabelEncoder
#binary
temp=df.iloc[:,0:1].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["school"]=temp
temp=df.iloc[:,1:2].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["sex"]=temp
temp=df.iloc[:,3:4].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["address"]=temp
temp=df.iloc[:,4:5].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["famsize"]=temp
temp=df.iloc[:,5:6].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["Pstatus"]=temp
temp=df.iloc[:,15:16].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["schoolsup"]=temp
temp=df.iloc[:,16:17].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["famsup"]=temp
temp=df.iloc[:,17:18].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["paid"]=temp
temp=df.iloc[:,18:19].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["activities"]=temp
temp=df.iloc[:,19:20].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["nursery"]=temp
temp=df.iloc[:,20:21].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["higher"]=temp
temp=df.iloc[:,21:22].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["internet"]=temp
temp=df.iloc[:,22:23].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["romantic"]=temp
#numeric

Mjob = pd.get_dummies(df["Mjob"])
Fjob = pd.get_dummies(df["Fjob"])
reason = pd.get_dummies(df["reason"])
guardian = pd.get_dummies(df["guardian"])
Mjob = Mjob.rename(columns={"teacher":"Mother_teacher","at_home": "Mother_at_home", "health" : "Mother_health", "services":"Mother_services","other":"Mother_other"})
Fjob = Fjob.rename(columns={"teacher":"Father_teacher","at_home": "Father_at_home", "health" : "Father_health", "services":"Fther_services","other":"Father_other"})
guardian = guardian.rename(columns={"other":"g_other"})
df=pd.concat([df,Mjob,Fjob,reason,guardian],join='inner',axis=1)
df.drop(["Mjob","Fjob","reason","guardian"],axis=1,inplace=True)
df


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Father_other,Fther_services,Father_teacher,course,home,other,reputation,father,mother,g_other
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,1,0,19,0,0,1,2,3,1,3,...,1,0,0,1,0,0,0,0,1,0
645,1,0,18,1,1,1,3,1,1,2,...,0,1,0,1,0,0,0,0,1,0
646,1,0,18,1,0,1,1,1,2,2,...,1,0,0,1,0,0,0,0,1,0
647,1,1,17,1,1,1,3,1,2,1,...,0,1,0,1,0,0,0,0,1,0


In [5]:
#import os
#df.to_csv("new_student_por.csv")

In [6]:
df.groupby("PASS")["PASS"].count()

PASS
0    100
1    549
Name: PASS, dtype: int64

In [7]:
#özellik ve etiketlerin ayrılması
x=df.drop("PASS",axis=1)
y=df["PASS"]

In [8]:
x.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid',
       'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'Mother_at_home', 'Mother_health', 'Mother_other', 'Mother_services',
       'Mother_teacher', 'Father_at_home', 'Father_health', 'Father_other',
       'Fther_services', 'Father_teacher', 'course', 'home', 'other',
       'reputation', 'father', 'mother', 'g_other'],
      dtype='object')

In [9]:
#SMOTE---https://towardsdatascience.com/applying-smote-for-class-imbalance-with-just-a-few-lines-of-code-python-cdf603e58688
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_res, y_res = smote.fit_resample(x, y)

In [10]:
X_res


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Father_other,Fther_services,Father_teacher,course,home,other,reputation,father,mother,g_other
0,0.0,0.0,18,1.0,0.0,0.0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0.0,0.0,17,1.0,0.0,1.0,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0.0,0.0,15,1.0,1.0,1.0,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0.0,0.0,15,1.0,0.0,1.0,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0.0,0.0,16,1.0,0.0,1.0,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,1.0,0.0,17,0.0,0.0,1.0,1,1,1,1,...,1,0,0,0,0,1,0,0,0,0
1094,0.085286,0.0,17,0.0,0.0,1.0,2,1,1,1,...,1,0,0,0,0,0,0,0,1,0
1095,1.0,0.0,19,0.926049,0.0,0.926049,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1096,1.0,0.0,19,0.059804,0.0,0.059804,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_res = X_res.round(0).astype(int) #ondalıklı değerleri yuvarlama

In [12]:
X_res

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Father_other,Fther_services,Father_teacher,course,home,other,reputation,father,mother,g_other
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,1,0,17,0,0,1,1,1,1,1,...,1,0,0,0,0,1,0,0,0,0
1094,0,0,17,0,0,1,2,1,1,1,...,1,0,0,0,0,0,0,0,1,0
1095,1,0,19,0,0,0,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1096,1,0,19,0,0,0,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_res

0       1
1       1
2       1
3       1
4       1
       ..
1093    0
1094    0
1095    0
1096    0
1097    0
Name: PASS, Length: 1098, dtype: int64

In [28]:
#smote=pd.concat([X_res,y_res],join="inner",axis=1)
#smote.to_csv("smote_por.csv")


In [32]:
def DT_clf(x,y):
    clf = DecisionTreeClassifier(random_state=42)
    return np.mean(cross_val_score(clf, x, y, cv=10))

def RF_clf(x,y):
    clf = RandomForestClassifier(random_state=42)
    return np.mean(cross_val_score(clf, x, y, cv=10))

def SVC_clf(x,y):
    clf = SVC(kernel='linear', random_state = 0)
    return np.mean(cross_val_score(clf, x, y, cv=10))

def Ada_clf(x,y):
    clf = AdaBoostClassifier()
    return np.mean(cross_val_score(clf, x, y, cv=10))

In [33]:
print("SMOTE uygulanmadan önce k=10 cross validation ile: ")
print("Decission Tree     :",DT_clf(x,y))
print("Random Forest      :",RF_clf(x,y))
print("Support Vector Clf :",SVC_clf(x,y))
print("Adaboost           :",Ada_clf(x,y))

SMOTE uygulanmadan önce k=10 cross validation ile: 
Decission Tree     : 0.8997836538461538
Random Forest      : 0.9167307692307693
Support Vector Clf : 0.8936057692307692
Adaboost           : 0.9090144230769232


In [34]:
print("SMOTE sonrası k=10 cross validation ile: ")
print("Decission Tree     :",DT_clf(X_res, y_res))
print("Random Forest      :",RF_clf(X_res, y_res))
print("Support Vector Clf :",SVC_clf(X_res, y_res))
print("Adaboost           :",Ada_clf(X_res, y_res))

SMOTE sonrası k=10 cross validation ile: 
Decission Tree     : 0.9380316930775645
Random Forest      : 0.9543619683069224
Support Vector Clf : 0.9307839866555463
Adaboost           : 0.9298665554628858


In [36]:
def DT_clf_holdout(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train,y_train)
    results = clf.predict(X_test)
    return accuracy_score(y_test, results)

def RF_clf_holdout(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train,y_train)
    results = clf.predict(X_test)
    return accuracy_score(y_test, results)

def SVC_clf_holdout(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
    clf = SVC(kernel='linear', random_state = 0)
    clf.fit(X_train, y_train)
    return clf.score(X_train, y_train)

def Ada_clf_holdout(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
    clf = AdaBoostClassifier()
    clf.fit(X_train, y_train)
    return clf.score(X_train,y_train)

--------------SMOTE uygulanmadan önce .30 holdout ile------------------

In [37]:
print("SMOTE uygulanmadan önce k=10 cross validation ile: ")
print("Decission Tree     :",DT_clf_holdout(x,y))
print("Random Forest      :",RF_clf_holdout(x,y))
print("Support Vector Clf :",SVC_clf_holdout(x,y))
print("Adaboost           :",Ada_clf_holdout(x,y))

SMOTE uygulanmadan önce k=10 cross validation ile: 
Decission Tree     : 0.9333333333333333
Random Forest      : 0.9333333333333333
Support Vector Clf : 0.9647577092511013
Adaboost           : 0.973568281938326


---------------SMOTE uygulandıktan sonra------------

In [38]:
print("SMOTE uygulanmadan önce k=10 cross validation ile: ")
print("Decission Tree     :",DT_clf_holdout(X_res, y_res))
print("Random Forest      :",RF_clf_holdout(X_res, y_res))
print("Support Vector Clf :",SVC_clf_holdout(X_res, y_res))
print("Adaboost           :",Ada_clf_holdout(X_res, y_res))

SMOTE uygulanmadan önce k=10 cross validation ile: 
Decission Tree     : 0.9393939393939394
Random Forest      : 0.9757575757575757
Support Vector Clf : 0.9635416666666666
Adaboost           : 0.9778645833333334
